# Code to produce the cutouts for the MIRI data (DONE)

Let's start with the imports

In [1]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

Load the catalogue and obtain IDs and coordinates of each galaxy

In [3]:
# Load target catalogue
with fits.open('./cat_targets.fits') as catalog_hdul:
    catalog_hdul.info()
    cat_data = catalog_hdul[1].data  # Extract data from table
    cat_header = catalog_hdul[1].header
    ids = cat_data['id']
    ra = cat_data['ra']
    dec = cat_data['dec']  


Filename: ./cat_targets.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     61   151R x 23C   [K, D, D, D, D, D, K, 4A, K, D, D, D, K, D, D, D, D, D, D, D, D, K, K]   


AttributeError: recarray has no attribute rows

# Determine cutout dimensions by converting pixels

Now let's inspect Amirs NIRCam images to see what dimensions he chose:

In [12]:
nircam_amir = '/home/bpc/University/master/Red_Cardinal/NIRCam/Amir/7102_F090W_cutout.fits'
nircam_tile = '/home/bpc/University/master/Red_Cardinal/NIRCam/COSMOS-1-Tile-24/jw01837-o039_t043_nircam_clear-f090w_i2d.fits'

# Load the NIRCam FITS file
with fits.open(nircam_amir) as nircam_hdul:
    nircam_hdul.info()
    nircam_data = nircam_hdul[1].data
    nircam_header = nircam_hdul[1].header
    wcs = WCS(nircam_header)


Filename: /home/bpc/University/master/Red_Cardinal/NIRCam/Amir/7102_F090W_cutout.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     377   ()      
  1  SCI           1 ImageHDU        81   (167, 167)   float32   
  2  ERR           1 ImageHDU        81   (167, 167)   float32   
  3  WHT           1 ImageHDU        81   (167, 167)   float32   
  4  EXP           1 ImageHDU        81   (167, 167)   float32   


Set OBSGEO-B to   -26.375750 from OBSGEO-[XYZ].
Set OBSGEO-H to 1600118801.710 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Let's look at the NIRCam documentation:

![NIRCam field of view and pixel sizes](./NIRCam_FOV.png)

So we find that in the short filters we have 0.031"/pix and Amir used 167 x 167 pixels. So we know that the total FOV is around 63.74" x 63.74". Now we need to translate this to MIRI:

![MIRI FOV](./MIRI_FOV.jpeg)
![MIRI FOV](./MIRI_pixel_sizes.png)

So takin this conversion of pixels to angular resolution we find that MIRI pixels are on average about 3.5 times larger than NIRCam pixels, so we just need to divide 167 by 3.55, which leaves us with ~48 pixels in MIRI.

# Create cutouts for PRIMER

In [13]:

# Define cutout size (in pixels)
cutout_size = (48, 48)

counts = 0
total = len(ra)

# Load datasets
primer = '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER/'
primer_data = glob.glob(os.path.join(primer, "*.fits"))
print(f"Found {len(primer_data)} FITS files.")

# Create output directory
output_dir = "./cutouts"

# Load the MIRI FITS file
for data in primer_data:
    with fits.open(data) as miri_hdul:
        miri_data = miri_hdul[1].data
        miri_header = miri_hdul[1].header
        wcs = WCS(miri_header)

    if "f770w" in data: filter = "F770W"
    if "f1800w" in data: filter = "F1800W"
    
    # Loop through the galaxies
    for i in range(len(ra)):
        target_coord = SkyCoord(ra[i], dec[i], unit=(u.deg, u.deg))

        # Check if the galaxy is inside the mapped region
        x, y = wcs.world_to_pixel(target_coord)
        if (0 <= x < miri_data.shape[1]) and (0 <= y < miri_data.shape[0]):
            
            # Extract the cutout
            cutout = Cutout2D(miri_data, target_coord, cutout_size, wcs=wcs, mode='partial')

            # Check if galaxy is visible
            nans = np.isnan(cutout.data).sum()
            nan_ratio = nans/cutout.data.size
            
            if nan_ratio < 0.4: 

                # Save the cutout
                plt.figure(figsize=(6, 6))
                plt.imshow(cutout.data, origin='lower', cmap='gray')
                plt.colorbar()
                plt.title(f'PRIMER: {ids[i]} at {filter}')

                png_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_primer.png")
                plt.savefig(png_filename)
                plt.close()
                
                counts += 1
                
                # Create a new FITS HDU list to store cutout data
                hdu_list = fits.HDUList()
                hdu_list.append(fits.PrimaryHDU(header=miri_hdul[0].header))
                hdu_list.append(fits.ImageHDU(data=cutout.data, header=cutout.wcs.to_header()))
                
                for ext in range(2, len(miri_hdul)):
                    hdu_list.append(fits.ImageHDU(data=miri_hdul[ext].data, header=miri_hdul[ext].header))

                # Save the cutout FITS file
                fits_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_primer.fits")
                hdu_list.writeto(fits_filename, overwrite=True)
                    
                #print(f"Saved: object {ids[i]} with filter {filter}")
                
            else:
                #print(f"Skipping cutout for target {ids[i]} with filter {filter}: Too many nans in frame")    
                continue
        else:
            #print(f"Skipping target {ids[i]}: Outside mapped region")
            continue
print(f"Produced cutouts for {counts} of {total} galaxies in the catalogue.")


Found 4 FITS files.


Set DATE-AVG to '2023-05-14T18:14:40.253' from MJD-AVG.
Set DATE-END to '2023-05-26T19:47:42.198' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -13.409758 from OBSGEO-[XYZ].
Set OBSGEO-H to 1412220181.737 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-01-01T12:00:51.883' from MJD-AVG.
Set DATE-END to '2023-01-06T04:05:17.902' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.121531 from OBSGEO-[XYZ].
Set OBSGEO-H to 1684160522.172 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-05-14T17:30:54.860' from MJD-AVG.
Set DATE-END to '2023-05-26T19:03:43.033' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -13.386215 from OBSGEO-[XYZ].
Set OBSGEO-H to 1411888769.318 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-01-01T15:58:38.112' from MJD-AVG.
Set DATE-END to '2023-01-06T04:49:11.500' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.112615 from OBSGEO-[XYZ].
Set OBSGEO-H to 1684267574.210 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Produced cutouts for 104 of 151 galaxies in the catalogue.


# Create cutouts for COSMOS-Web

In [14]:

# Define cutout size (in pixels)
cutout_size = (48, 48)

# Load datasets
cweb = "/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web/"
cweb_data = glob.glob(os.path.join(cweb, "*.fits"))
print(f"Found {len(cweb_data)} FITS files.")

# Create output directory
output_dir = "./cutouts"
os.makedirs(output_dir, exist_ok=True)

counts = 0
total = len(ra)

for data in cweb_data:
    with fits.open(data) as miri_hdul:
        miri_data = miri_hdul[1].data
        miri_header = miri_hdul[1].header
        wcs = WCS(miri_header)

    # Loop over all galaxies first
    for i in range(len(ra)):
        target_coord = SkyCoord(ra[i], dec[i], unit=(u.deg, u.deg))
        found = False  # Track if the galaxy was mapped

        filter = "F770W"

        # Check if the galaxy is inside the mapped region
        x, y = wcs.world_to_pixel(target_coord)
        if (0 <= x < miri_data.shape[1]) and (0 <= y < miri_data.shape[0]):
            found = True  # The galaxy was mapped

            # Extract the cutout
            cutout = Cutout2D(miri_data, target_coord, cutout_size, wcs=wcs, mode="partial")

            # Ensure cutout is valid
            if cutout.data is None or cutout.data.size == 0:
                #print(f"Skipping {ids[i]}: Empty cutout")
                continue

            # Check for NaN ratio
            nans = np.isnan(cutout.data).sum()
            nan_ratio = nans / cutout.data.size

            if nan_ratio < 0.4:
                # Save PNG preview
                plt.figure(figsize=(6, 6))
                plt.imshow(cutout.data, origin="lower", cmap="gray")
                plt.colorbar()
                plt.title(f'COSMOS-Web: {ids[i]} at {filter}')

                png_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_cweb.png")
                plt.savefig(png_filename)
                plt.close()

                counts += 1

                # Save the cutout FITS file
                hdu_list = fits.HDUList()
                hdu_list.append(fits.PrimaryHDU(header=miri_hdul[0].header))
                hdu_list.append(fits.ImageHDU(data=cutout.data, header=cutout.wcs.to_header()))

                for ext in range(2, len(miri_hdul)):
                    if miri_hdul[ext].data is not None:
                        hdu_list.append(fits.ImageHDU(data=miri_hdul[ext].data, header=miri_hdul[ext].header))

                fits_filename = os.path.join(output_dir, f"{ids[i]}_{filter}_cutout_cweb.fits")
                hdu_list.writeto(fits_filename, overwrite=True)

                #print(f"Saved: {ids[i]} at {filter}")
            else:
                #print(f"Skipping {ids[i]}: Too many NaNs ({nan_ratio:.2f})")
                continue
        else:
            continue  # Galaxy is outside the mapped region

    if not found:
        #print(f"Skipping {ids[i]}: Not found in any FITS file")
        continue
    
print(f"Produced cutouts for {counts} of {total} galaxies in the catalogue.")


Found 164 FITS files.


Set DATE-AVG to '2024-01-07T00:33:34.781' from MJD-AVG.
Set DATE-END to '2024-01-07T01:03:43.107' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.978885 from OBSGEO-[XYZ].
Set OBSGEO-H to 1686159123.800 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-12-30T22:51:50.121' from MJD-AVG.
Set DATE-END to '2023-12-30T23:21:48.712' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.880269 from OBSGEO-[XYZ].
Set OBSGEO-H to 1695431085.486 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-04-10T23:36:36.346' from MJD-AVG.
Set DATE-END to '2023-04-11T00:06:42.952' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     1.268650 from OBSGEO-[XYZ].
Set OBSGEO-H to 1267772074.933 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE-AVG to '2023-04-11T00:54:12.637' from MJD-AVG.
Set DATE-END to '2023-04-11T01:24:14.020' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     1.216279 from OBSGEO-[XYZ].
Set OBSGEO-H to 1268089069.933 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set DATE

Produced cutouts for 67 of 151 galaxies in the catalogue.


Set DATE-AVG to '2024-01-04T22:14:55.166' from MJD-AVG.
Set DATE-END to '2024-01-04T22:44:57.604' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to     8.884067 from OBSGEO-[XYZ].
Set OBSGEO-H to 1689382539.343 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


So let's check what's wrong with the cutouts that have more than 80% nan entries in our final plots.

problem_galaxy = './cutouts/8323_F770W_cutout_cweb.fits'

with fits.open(problem_galaxy) as prob_hdul:
    prob_data = prob_hdul[1].data
    prob_header = prob_hdul[1].header
    wcs = WCS(prob_header)

nan_count = np.isnan(prob_data).sum()
nan_ratio = nan_count / prob_data.size
    
print(nan_count)
print(prob_data.size)
print(nan_ratio)
#print(prob_data)

nan_count = np.sum(np.isnan(prob_data) | (prob_data <= 0))
valid_count = np.count_nonzero(~np.isnan(prob_data))
nan_ratio = 1 - (valid_count / prob_data.size)

print(nan_ratio)
plt.imshow(np.isnan(prob_data), cmap='gray', origin='lower')
plt.title('NaN mask')
plt.show()

